mnist分類学習（CPU）
chainer公式exampleより改変
https://github.com/chainer/chainer

In [ ]:
import argparse
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions

# Network definition

In [ ]:
class MLP(chainer.Chain):

    def __init__(self, n_units, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            # the size of the inputs to each layer will be inferred
            self.l1 = L.Linear(None, n_units)  # n_in -> n_units
            self.l2 = L.Linear(None, n_units)  # n_units -> n_units
            self.l3 = L.Linear(None, n_out)  # n_units -> n_out

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)

# 引数

In [ ]:
class Args(object):
    def __init__(self):
        self.batchsize = 100
        self.epoch = 2
        self.frequency = -1
        self.gpu = -1
        self.out = "result-chainer-mnist"
        self.resume = ' '
        self.unit = 50

In [ ]:
args = Args()
print('GPU: {}'.format(args.gpu))
print('# unit: {}'.format(args.unit))
print('# Minibatch-size: {}'.format(args.batchsize))
print('# epoch: {}'.format(args.epoch))
print('')

学習のためのニューラルネットワークをセット。
分類機のソフトマックス交差エントロピー誤差と精度のをループ毎に出力。

In [ ]:
model = L.Classifier(MLP(args.unit, 10))

# Setup an optimizer

In [ ]:
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

# Load the MNIST dataset

In [ ]:
train, test = chainer.datasets.get_mnist()

train_iter = chainer.iterators.SerialIterator(train, args.batchsize)
test_iter = chainer.iterators.SerialIterator(test, args.batchsize, repeat=False, shuffle=False)

# Set up a trainer

In [ ]:
updater = training.updaters.StandardUpdater(train_iter, optimizer, device=args.gpu)
trainer = training.Trainer(updater, (args.epoch, 'epoch'), out=args.out)

# Evaluate the model with the test dataset for each epoch

In [ ]:
trainer.extend(extensions.Evaluator(test_iter, model, device=args.gpu))

Dump a computational graph from 'loss' variable at the first iteration
The "main" refers to the target link of the "main" optimizer.

In [ ]:
trainer.extend(extensions.dump_graph('main/loss'))

# Take a snapshot for each specified epoch

In [ ]:
frequency = args.epoch if args.frequency == -1 else max(1, args.frequency)
trainer.extend(extensions.snapshot(), trigger=(frequency, 'epoch'))

# ログのレポートを作成o

In [ ]:
trainer.extend(extensions.LogReport())

In [ ]:
trainer.extend(
    extensions.PlotReport(['main/loss', 'validation/main/loss'],
                          'epoch', file_name='loss.png'))
trainer.extend(
    extensions.PlotReport(
        ['main/accuracy', 'validation/main/accuracy'],
        'epoch', file_name='accuracy.png'))

In [ ]:
# Print selected entries of the log to stdout
# Here "main" refers to the target link of the "main" optimizer again, and
# "validation" refers to the default name of the Evaluator extension.
# Entries other than 'epoch' are reported by the Classifier link, called by
# either the updater or the evaluator.
trainer.extend(extensions.PrintReport(
    ['epoch', 'main/loss', 'validation/main/loss',
     'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

In [ ]:
# Print a progress bar to stdout
trainer.extend(extensions.ProgressBar())

In [ ]:
trainer.run()